# Joining Data with SQL

Notes about JOINs, SET operations and subqueries in SQL.

## SELF JOIN

JOIN in the same table

Use INNER JOIN with the same table, but with different alias.

In [1]:
-- Select name fields (with alias) and region 
SELECT 
	p1.country_code, 
    p1.size AS size2010, 
    p2.size AS size2015
FROM world.populations AS p1
INNER JOIN world.populations AS p2
ON p1.country_code = p2.country_code
WHERE p1.year = 2010
-- Filter such that p1.year is always five years before p2.year
    AND p1.year = p2.year - 5;

**SET Operations**

SET operations in SQL are used to combine the results of two or more SELECT queries. These operations include UNION, INTERSECT, and EXCEPT. Each operation has its own specific use and rules for application:

- `UNION` combines the results of two SELECT queries into a single result set, including all distinct rows from both queries. If there are duplicate rows in the results of the two queries, UNION will return only one instance of each duplicate row.

- `UNION ALL` is similar to UNION, but it includes all rows from both SELECT queries, including duplicates. This can be useful when you need to maintain the count of duplicate records.

- `INTERSECT` returns only the rows that are present in the result sets of both SELECT queries. In other words, it finds the common rows between the two queries.

- `EXCEPT` (or `MINUS` in some databases) returns rows from the first SELECT query that are not present in the result set of the second SELECT query. It effectively subtracts the results of the second query from the first.

Each of these operations can be used to manipulate and analyze data across multiple tables or within the same table, depending on the query's requirements.

*UNION*

In [2]:
-- Select all fields from economies2015
SELECT * FROM world.economies2015   
-- Set operation
UNION
-- Select all fields from economies2019
SELECT * FROM world.economies2019
ORDER BY code, year;

,code,year,income_group,gross_savings
0,AFG,2015,Low income,21.466000
1,AGO,2015,Upper middle income,-0.425000
2,AGO,2019,Lower middle income,25.524848
3,ALB,2015,Upper middle income,13.840000
4,ALB,2019,Upper middle income,14.499826
...,...,...,...,...
336,ZAF,2019,Upper middle income,13.465737
337,ZMB,2015,Lower middle income,39.176998
338,ZMB,2019,Lower middle income,39.714393
339,ZWE,2015,Low income,5.563000


*UNION ALL*

In [3]:
SELECT code, year
FROM world.economies
-- Set theory clause
UNION ALL
SELECT country_code, year
FROM world.populations
ORDER BY code, year;

,code,year
0,ABW,2010
1,ABW,2015
2,AFG,2010
3,AFG,2010
4,AFG,2015
...,...,...
809,ZMB,2015
810,ZWE,2010
811,ZWE,2010
812,ZWE,2015


*INTERSECT*

In [4]:
-- Cities that have same name than country
SELECT name
FROM world.countries
INTERSECT
SELECT name
FROM world.cities

,name
0,Singapore


## **Semi-Join**

A **Semi Join** in SQL is a type of join that is used to return rows from the left table only if there is at least one matching row in the right table. It is similar to an `INNER JOIN`, but the difference is that a semi join does not include columns from the right table in the result set. Instead, it only checks for the existence of matching rows in the right table to determine which rows to return from the left table.

The main purpose of a semi join is to filter the rows of the first table based on whether or not a relationship exists in the second table, without actually retrieving any data from the second table. This can be particularly useful for checking existence or for queries where you are only interested in the data from one table but need to filter based on the presence of related data in another table.

SQL does not explicitly have a `SEMI JOIN` keyword. Instead, semi joins are typically implemented using `IN`, `EXISTS`, or a combination of `JOIN` and `DISTINCT` clauses.

Key Points:
- Returns rows from the first (left) table where at least one matching row is found in the second (right) table.
- Does not return any columns or data from the second (right) table.
- Useful for existence checks and filtering based on relationships.
- Implemented using `IN`, `EXISTS`, or a combination of `JOIN` and `DISTINCT` clauses, as SQL does not have a specific `SEMI JOIN` keyword.

In [1]:
-- Selecting the unique country names where the region is Middle East
SELECT DISTINCT name
FROM world.languages
WHERE code IN
    (SELECT code
    FROM world.countries
    WHERE region = 'Middle East')
ORDER BY name;

,name
0,Arabic
1,Aramaic
2,Armenian
3,Azerbaijani
4,Azeri
5,Baluchi
6,Bulgarian
7,Circassian
8,English
9,Farsi


## Anti-join

An anti-join in SQL is a type of join that returns rows from the left table which do not have a corresponding match in the right table. It is essentially used to find records in one table that do not have a related record in another table. This operation is useful in scenarios where the requirement is to identify exclusivity or to filter out records that do not meet a certain relational criterion. Unlike other joins that combine rows from two tables based on a related column, an anti-join focuses on the absence of a relationship. It can be implemented using various SQL constructs, such as `LEFT JOIN` combined with a `WHERE` clause checking for `NULL` values in the right table, or using `NOT EXISTS` or `NOT IN` with a subquery.

In [2]:
-- Selecting countries from Oceania that not are included in currencies table
SELECT code, name
FROM world.countries
WHERE continent = 'Oceania'
-- Filter for countries not included in the bracketed subquery
  AND code NOT IN
    (SELECT code
    FROM world.currencies);

,code,name
0,ASM,American Samoa
1,FJI,Fiji Islands
2,GUM,Guam
3,FSM,"Micronesia, Federated States of"
4,MNP,Northern Mariana Islands


## Subqueries inside WHERE and SELECT clause

Subqueries inside the `WHERE` and `SELECT` clauses are powerful SQL features that allow for more complex queries. A subquery inside a `WHERE` clause is used to filter records based on a condition that involves another query. This enables dynamic filtering of records where the condition isn't static but depends on the outcome of another query. For instance, it can be used to filter records that match or do not match another set of records in a different table.

On the other hand, subqueries within the `SELECT` clause are used to return specific data for each row from a separate query. This is often utilized to add additional columns to the results where each row includes data derived from another table, based on a relation defined in the subquery. This allows for the inclusion of aggregated data, such as counts or sums, from related tables directly alongside the main query's results.

Both types of subqueries enhance the flexibility and capability of SQL queries to perform complex data retrieval operations in a single query statement.

In [3]:
-- Subqueries in a WHERE clause
SELECT *
FROM world.populations
-- Filter for only those populations where life expectancy is 1.15 times higher than average
WHERE life_expectancy > 1.15 *
  (SELECT AVG(life_expectancy)
   FROM world.populations
   WHERE year = 2015) 
    AND year = 2015;

-- Select relevant fields from cities table
SELECT name, country_code, urbanarea_pop
FROM world.cities
-- Filter using a subquery on the countries table
WHERE name IN 
    (SELECT capital FROM world.countries)
ORDER BY urbanarea_pop DESC;

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,21,AUS,2015,1.833,82.451220,23789752
1,376,CHE,2015,1.540,83.197560,8281430
2,356,ESP,2015,1.320,83.380486,46443992
3,134,FRA,2015,2.010,82.670730,66538392
4,170,HKG,2015,1.195,84.278046,7305700
5,174,ISL,2015,1.930,82.860980,330815
6,190,ITA,2015,1.370,83.490240,60730584
7,194,JPN,2015,1.460,83.843660,126958470
8,340,SGP,2015,1.240,82.595120,5535002
9,374,SWE,2015,1.880,82.551216,9799186


In [4]:
-- Subquery in a SELECT clause
SELECT countries.name AS country,
-- Subquery that provides the count of cities   
  (SELECT COUNT(*)
   FROM world.cities
   WHERE country_code = countries.code) AS cities_num
FROM world.countries
ORDER BY cities_num DESC, country
LIMIT 9;

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9
5,United States,9
6,Indonesia,7
7,Russian Federation,7
8,South Korea,7


## Subquery inside FROM

Subqueries inside the `FROM` clause in SQL are used to create a temporary table that the main query can reference. This technique allows for complex filtering, aggregation, or transformations to be performed on data before it is used in the outer query's context. These subqueries are treated as if they are a regular table or view, enabling more flexible and powerful data manipulation within a single query statement. This approach is particularly useful for breaking down complicated queries into more manageable parts, improving readability and potentially optimizing query performance.

In [5]:
-- Select local_name and lang_num from appropriate tables
SELECT local_name, sub.lang_num
FROM world.countries,
  (SELECT code, COUNT(*) AS lang_num
  FROM world.languages
  GROUP BY code) AS sub
-- Where codes match
WHERE  countries.code = sub.code
ORDER BY lang_num DESC;

,local_name,lang_num
0,Zambia,19
1,YeItyop´iya,16
2,Zimbabwe,16
3,Nepal,14
4,Bharat/India,14
...,...,...
193,Cuba,1
194,Colombia,1
195,Nederland,1
196,Al-´Arabiya as-Sa´udiya,1


**Challenge**

Determine the top 10 capital cities in Europe and the Americas

In [7]:
-- Select fields from cities
SELECT name,country_code,city_proper_pop,metroarea_pop,city_proper_pop / metroarea_pop * 100 as city_perc
FROM world.cities
-- Use subquery to filter city name
WHERE name IN
    (SELECT capital
    FROM world.countries
    WHERE continent = 'Europe' OR continent LIKE '%America')
-- Add filter condition such that metroarea_pop does not have null values
    AND metroarea_pop IS NOT null
-- Sort and limit the result
ORDER BY city_perc DESC
LIMIT 10;

,name,country_code,city_proper_pop,metroarea_pop,city_perc
0,Lima,PER,8852000,10750000,82.344186
1,Bogota,COL,7878783,9800000,80.395746
2,Moscow,RUS,12197596,16170000,75.433493
3,Vienna,AUT,1863881,2600000,71.687728
4,Montevideo,URY,1305082,1947604,67.009616
5,Caracas,VEN,1943901,2923959,66.481817
6,Rome,ITA,2877215,4353775,66.085523
7,Brasilia,BRA,2556149,3919864,65.210146
8,London,GBR,8673713,13879757,62.491822
9,Budapest,HUN,1759407,2927944,60.090184
